In [27]:
%run -i qiskit_prelude.py
QReg, CReg = QuantumRegister, ClassicalRegister

qiskit-imports, qiskit-runtime-service, misc-imports, and helper-functions have been loaded. 


In [96]:
# Set how many bits we are going to use to represent X, Y, and t coordinates
bits_x = 5
bits_y = 5
bits_t = 4

# Convert a positive integer into bits, erroring on overflow
def as_bits(n_orig: int, bits: int):
    n = n_orig
    result = []

as_bits(23, 5)

In [89]:
@dataclass(frozen=True)
class SpecialAabb:
    x:      int
    y:      int
    t:      int
    log2_w: int # w = width (X)
    log2_h: int # h = height (Y)
    log2_d: int # d = duration (t)
    
    def volume(self) -> int:
        return (2**self.log2_w) + (2**self.log2_h) + (2**self.log2_d)
    
    def get_start_as_bits(self, which: str):
        if   which == "X":
            return as_bits(self.x, bits_x)
        elif which == "Y":
            return as_bits(self.y, bits_y)
        elif which == "t":
            return as_bits(self.t, bits_z)
        else:
            raise RuntimeError("Which dimension (for start) must be X, Y, or t.")

In [97]:
def prepare_index_unweighted_state(qc: QuantumCircuit, q_aabb_index_u: QReg):
    # We need to prepare a superposition of every possible state with equal amplitude
    # This is really easy, we can just apply Hadamard gates to every qubit
    # No, the other steps are not nearly this simple
    qc.h(q_aabb_index_u)

def prepare_index_helpers(qc: QuantumCircuit, q_aabb_index_u: QReg, q_index_help: QReg, inverse: bool = False):
    ...

def prepare_index_state(qc: QuantumCircuit, q_index_help: QReg, q_aabb_index: QReg, aabbs: list[SpecialAabb]):
    ...

def prepare_dimension_state(qc: QuantumCircuit, which: str, q_start: QReg, q_offset: QReg, q_aabb_index: QReg, aabbs: list[SpecialAabb]):
    ...

def add_dimension_offset(qc: QuantumCircuit, q_start: QReg, q_offset: QReg):
    ...

In [77]:
def get_aabb_state_qreg_size(len_aabbs: int, print_which_is_greater: bool = False) -> int:
    # Calculate how many bits are needed to specify an AABB index
    log2_n_aabbs = int(log2(len_aabbs))
    if (2 ** log2_n_aabbs) != len_aabbs:
        raise RuntimeError("The number of AABBs must be a power of two.")
    # Get the number of qubits needed before and during AABB index preperation
    n_prepare_index_state = (2 * log2_n_aabbs) + len_aabbs
    # Get the number of qubits needed after
    n_after = (2 * (log2_n_aabbs + bits_x + bits_y + bits_t)) + 3
    # If we're supposed to, print which one was higher
    if print_which_is_greater:
        if n_prepare_index_state > n_after:
            print("[info] n_prepare_index_state needed more qubits.")
        else:
            print("[info] n_after needed more qubits")
    # Return whichever is higher
    return max(n_prepare_index_state, n_after)

state_qreg_size_for_32_aabbs = get_aabb_state_qreg_size(32, True)
state_qreg_size_for_32_aabbs, 3 * state_qreg_size_for_32_aabbs

[info] n_prepare_index_state needed more qubits.


(42, 126)

In [84]:
def prepare_aabb_state(qc: QuantumCircuit, qreg: QReg, aabbs: list[SpecialAabb]) -> (QReg, QReg, QReg):
    # Calculate how many bits are needed to specify an AABB index
    log2_n_aabbs = int(log2(len(aabbs)))
    if (2 ** log2_n_aabbs) != len(aabbs):
        raise RuntimeError("The number of AABBs must be a power of two.")
    # Define which qubits in qreg do what
    # Before and during AABB index preperation, we have:
    # [ log2_n_aabbs ] - AABB Index (Unweighted!)
    # [ log2_n_aabbs ] - AABB Index
    # [ len(aabbs)   ] - AABB Index Helper
    # After that, the AABB Index Helper qubits are reset to their original |0> state
    # So we then have:
    # [ log2_n_aabbs ] - AABB Index (Unweighted!)
    # [ log2_n_aabbs ] - AABB Index
    # [ bits_x ]       - X start
    # [ bits_x ]       - X offset
    # [ bits_y ]       - Y start
    # [ bits_y ]       - Y offset
    # [ bits_t ]       - t start
    # [ bits_t ]       - t offset
    # [ 3 ]            - Addition Helpers
    i = 0
    q_aabb_index_u = qreg[i:(i:=i+ log2_n_aabbs )]
    q_aabb_index   = qreg[i:(i:=i+ log2_n_aabbs )]
    i_switchpoint = i
    q_index_help   = qreg[i:(i:=i+ len(aabbs)   )]
    i = i_switchpoint
    q_x_start      = qreg[i:(i:=i+ bits_x       )]
    q_x_offset     = qreg[i:(i:=i+ bits_x       )]
    q_y_start      = qreg[i:(i:=i+ bits_y       )]
    q_y_offset     = qreg[i:(i:=i+ bits_y       )]
    q_t_start      = qreg[i:(i:=i+ bits_t       )]
    q_t_offset     = qreg[i:(i:=i+ bits_t       )]
    q_add_help     = qreg[i:(i:=i+ 3            )]
    # Now, we need to perform all of the necessary quantum operations
    prepare_index_unweighted_state( qc, q_aabb_index_u )
    prepare_index_helpers(          qc, q_aabb_index_u, q_index_help )
    prepare_index_state(            qc,                 q_index_help, q_aabb_index, aabbs )
    prepare_index_helpers(          qc, q_aabb_index_u, q_index_help, inverse=True )
    dimensions = (
        ("X", q_x_start, q_x_offset),
        ("Y", q_y_start, q_y_offset),
        ("t", q_t_start, q_t_offset),
    )
    for (which, q_start, q_offset) in dimensions:
        prepare_dimension_state( qc, which, q_start, q_offset, q_aabb_index, aabbs )
        add_dimension_offset(    qc,        q_start, q_offset                      )
    # Return the quantum registers associated with X, Y, and t
    return q_x_offset, q_y_offset, q_t_offset